In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN, Dropout;

In [2]:
train_data = pd.read_csv('/kaggle/input/google-stock-data/GOOGL.csv')
train_data = train_data.drop('Adj Close',axis=1)

In [3]:
train_data.head()

,Date,Open,High,Low,Close,Volume
0,2004-08-19,50.050049,52.082081,48.028027,50.220219,44659096
1,2004-08-20,50.555557,54.594597,50.300301,54.209209,22834343
2,2004-08-23,55.430431,56.796799,54.579578,54.754753,18256126
3,2004-08-24,55.675674,55.855858,51.836838,52.487488,15247337
4,2004-08-25,52.532532,54.054054,51.991993,53.053055,9188602


In [4]:
train_data.isna().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [5]:
train_data.shape

(4431, 6)

In [6]:
train_col = train_data.iloc[:,1:2]

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_col = scaler.fit_transform(train_col)

In [15]:
scaled_train_col[:,:]

array([[1.36253641e-04],
       [3.06152003e-04],
       [1.94456941e-03],
       ...,
       [8.98173517e-01],
       [9.15657150e-01],
       [9.19001272e-01]])

In [9]:
x_train = np.array([scaled_train_col[i-50:i,0] for i in range(50,len(scaled_train_col))])
y_train = np.array([scaled_train_col[i,0] for i in range(50,len(scaled_train_col))])

In [10]:
x_train_new = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [11]:
x_train.shape

(4381, 50)

In [12]:
model = Sequential([
    Input(shape=(50,1)),
    SimpleRNN(units=50,return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=50,return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=50,return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=50,return_sequences=True),
    Dropout(0.2),
    Dense(units=1)
])

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 50, 50)         │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 50, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 50, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,801 (69.54 KB)

 Trainable params: 17,801 (69.54 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])
model.fit(x_train,y_train, epochs=100,batch_size=32)

Epoch 1/100


I0000 00:00:1730576318.878659      92 service.cc:145] XLA service 0x58ef13b0c230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730576318.878719      92 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  8/137 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.4906 - mean_squared_error: 0.4906

I0000 00:00:1730576325.105900      92 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_4', 32 bytes spill stores, 52 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1730576325.117061      92 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


137/137 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - loss: 0.2193 - mean_squared_error: 0.2193
Epoch 2/100
  4/137 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.0744 - mean_squared_error: 0.0744

I0000 00:00:1730576333.201884      94 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_5', 32 bytes spill stores, 52 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 4 bytes spill stores, 4 bytes spill loads



137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0589 - mean_squared_error: 0.0589
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0510 - mean_squared_error: 0.0510
Epoch 4/100
121/137 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0484 - mean_squared_error: 0.0484


KeyboardInterrupt

